<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebooks/blob/master/Signature_Line_Detection_RestFul_API_TransData_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF4~=1.27.0
!pip install pdf2image~=1.16.0
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install poppler-utils
#!pip install paddleocr
#!pip install paddlepaddle paddleocr
#!pip install Pillow==9.0.0 --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61246 sha256=f5ad9590aec0ffccc04ab1dbcf879bb626ec233c586dc581af316b0144493bd1
  Stored in directory: /root/.cache/pip/wheels/cd/0e/4a/e6f842a6035ccffff0dab29c39dd06c3427560a82783355a83
Successfully built PyPDF4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this ope

In [ ]:
import cv2
import base64
from PIL import Image
from io import BytesIO
import json
import numpy as np
import pytesseract

In [ ]:
custom_config = r'-l eng --psm 6'
def get_text_from_image_tessaract(image):        
    page_text = ""
    with BytesIO() as f:
            page_text = pytesseract.image_to_string(image, 
                                                    config= custom_config)      
    return page_text.strip()

In [ ]:
def lines_detection_from_image(img_path, min_length=10, max_length=30):
    # Load image, convert to grayscale, Otsu's threshold
    with open(img_path, "rb") as image_file:
        data = base64.b64encode(image_file.read())

    im = Image.open(BytesIO(base64.b64decode(data)))
    image = np.asarray(im)

    result = image.copy()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Detect horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
    detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    lines_dect = []

    for c in cnts:
        lines_dect.append(c)
        cv2.drawContours(result, [c], -1, (255,0,0), 2) # draw contours
    
    coordinates_points = []
    load_img = Image.open(img_path)
    for i in lines_dect:
        x,y,w,h = cv2.boundingRect(i) # getting rectangle coordinates from contours
        
        # Check if line length is within specified minimum and maximum length constraints
        if w >= min_length and w <= max_length:
            box = x-150,y,x+w,y+50 # box coordinates for cropping image
            crop_img = load_img.crop(box) # cropping the image
            text = get_text_from_image_tessaract(crop_img) # getting the text
            
            
            line_length = w  # Length of the horizontal line in pixels
            
            line_coordinates = {
                'coordinates':{
                "x": x,
                "y": y,
                "width": w,
                "height": h,
                'text': text,
                'length': line_length
                }
            }
            coordinates_points.append(line_coordinates)
    
    return len(lines_dect), coordinates_points


In [ ]:
def REST_API(total_lines, coordinates):
    json_response = {
    'lines_count': total_lines,
    'detection': coordinates,
    # "signature_type": {
    #     "text": {
    #         "signature":'',
    #         "date":''
    #         }
    #     }
    }

    json_response_ = json.dumps(json_response)
    return json_response_

In [ ]:
total_lines, coordinates = lines_detection_from_image('/content/signature-page-1.jpg', min_length=100, max_length=200)

In [ ]:
res = REST_API(total_lines, coordinates)
json.loads(res)

{'lines_count': 17,
 'detection': [{'coordinates': {'x': 1252,
    'y': 1821,
    'width': 184,
    'height': 1,
    'text': 'Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 1667,
    'width': 184,
    'height': 2,
    'text': 'Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 1476,
    'width': 184,
    'height': 1,
    'text': 'Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 1246,
    'width': 184,
    'height': 1,
    'text': 'Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 1054,
    'width': 184,
    'height': 1,
    'text': '- Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 862,
    'width': 184,
    'height': 2,
    'text': 'Date',
    'length': 184}},
  {'coordinates': {'x': 1252,
    'y': 671,
    'width': 184,
    'height': 1,
    'text': 'Date',
    'length': 184}}]}

## **Extracting Text**

In [ ]:
# IMAGE = '/content/document_page_z7QIGHs.png'
# load_img = Image.open(IMAGE)
# load_img

In [ ]:
# x = coordinates[2]['coordinates']['x']
# y = coordinates[2]['coordinates']['y']
# w = coordinates[2]['coordinates']['width']
# h = coordinates[2]['coordinates']['height']

# box = x-150,y,x+w,y+50
# box

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# img = cv2.imread("/content/document_page_z7QIGHs.png")
# crop = img[y:y+50, x-150:x+w]
# plt.imshow(crop, cmap='binary')
# plt.show()

In [ ]:
# img = Image.open(IMAGE)
# img2 = img.crop(box)
# img2

In [ ]:
# text = get_text_from_image_tessaract(crop)
# text

In [ ]:
# get_text_from_image_tessaract('r1.PNG')


In [ ]:
# get_text_from_image_tessaract('r2.PNG')


In [ ]:
# get_text_from_image_tessaract('r3.PNG')


In [ ]:
# get_text_from_image_tessaract('r4.PNG')
